# Preparation

In [ ]:
#@title
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#@title
import requests
import xml.etree.ElementTree as et
import pandas as pd
import datetime
from collections import OrderedDict
import os
import time
from google.colab import files

In [ ]:
# there are many html tags in abstract which hamper xml parsing
# list is taken from https://www.w3schools.com/html/html_formatting.asp
# but there  might still be some leftovers
'''
    <b> - Bold text
    <strong> - Important text
    <i> - Italic text
    <em> - Emphasized text
    <mark> - Marked text
    <small> - Smaller text
    <del> - Deleted text
    <ins> - Inserted text
    <sub> - Subscript text
    <sup> - Superscript text
'''
ignore_tags = ['i', 'b', 'sub', 'sup', 'em', 'strong', 'small', 'mark', 'del', 'ins']
def getXMLfromURL(url, params, ignore_tags = ignore_tags):
    res_text = requests.get(url, params).text
    for ignore_tag in ignore_tags:
        res_text = \
            res_text.replace('<'+ignore_tag+'>', '').replace('</'+ignore_tag+'>', '')
    root = et.fromstring(res_text.encode("utf-8"))
    # root = et.fromstring(res.text)
    return root

def getTextFromNode(root, path, fill_none = '', sep = ', ', concat = False):
    def _concat():
        return sep.join([elem.text if elem.text is not None else ' ' \
                         for elem in root.findall(path)]) 
    if (root.find(path) == None):
        return fill_none
    if (concat == True):
        return _concat()
    else:
        return root.find(path).text

''' some artcles do not have PubDate so use DateCompleted instead
def parsePublicationDate(article, sep = '/'):
    # parse publication date
    _yr = getTextFromNode(article, 'MedlineCitation/Article/Journal/JournalIssue/PubDate/Year')
    _mth = str(datetime.datetime.strptime(\
            getTextFromNode(article, 'MedlineCitation/Article/Journal/JournalIssue/PubDate/Month'), '%b').month)
    _d = getTextFromNode(article, 'MedlineCitation/Article/Journal/JournalIssue/PubDate/Day')
    dt = sep.join([_yr, _mth, _d])
    
    return dt
'''

def parseAuthors(article, sep = ', '):
    # parse authors
    authors = []
    for author in article.findall('MedlineCitation/Article/AuthorList/'):
        authors.append(getTextFromNode(author, 'ForeName') + ' ' +  getTextFromNode(author,'LastName'))
    return sep.join(authors)

def parseData(root):
    articleDics = []
    for article in root.iter('PubmedArticle'):
        # get article info
        articleDic = {
            'PMID'                    : getTextFromNode(article, 'MedlineCitation/PMID'),
            'JournalTitle'            : getTextFromNode(article, 'MedlineCitation/Article/Journal/Title'),
            'Title'                   : getTextFromNode(article, 'MedlineCitation/Article/ArticleTitle'),
            'Abstract'                : getTextFromNode(article, 'MedlineCitation/Article/Abstract/AbstractText', concat = True),
            'Publication Date'        : getTextFromNode(article, 'MedlineCitation/DateCompleted/Year'),
            'Chemical'                : getTextFromNode(article, 'MedlineCitation/ChemicalList/Chemical/NameOfSubstance', concat = True),
            'MeSH'                    : getTextFromNode(article, 'MedlineCitation/MeshHeadingList/MeshHeading/DescriptorName', concat = True),
            'doi'                     : getTextFromNode(article, 'MedlineCitation/Article/ELocationID'),
            'authors'                 : parseAuthors(article),
            'keywords'                : getTextFromNode(article, 'MedlineCitation/KeywordList/Keyword', concat = True),
        }
        articleDics.append(OrderedDict(articleDic))
    return articleDics

def pullData(term):
    search_params = {
    "db": SOURCE_DB,
    "term": term,
    "usehistory": "y",
    "retmode" : "xml",
    }
    
    root_search = getXMLfromURL(SEARCH_URL, params = search_params)
    
    print(f'query translation:')
    print(f'{root_search.find("QueryTranslation").text}')
    print(f'{root_search.find("Count").text} results found')
    
    fetch_params = {
        "db" : "pubmed",
        "query_key" : root_search.findtext("QueryKey"),
        "WebEnv" : root_search.findtext("WebEnv"),
        "retmode" : "xml",
    }
    root_res = getXMLfromURL(FETCH_URL, params = fetch_params)
    # printRecur(root_res)
    res_dict = parseData(root_res)
    res_df = pd.DataFrame.from_dict(res_dict)
    time.sleep(.5)
    return res_df
    
    
'''
# https://qiita.com/nomurasan/items/78d0174977bb61a52808
def xml2df(xml_data):
    root = ET.XML(xml_data) # element tree
    all_records = []
    for i, child in enumerate(root):
        record = {}
        for subchild in child:
            record[subchild.tag] = subchild.text
        all_records.append(record)
    df = pd.DataFrame(all_records)
    return df
'''

'''
# check structure of retrived data
# https://stackoverflow.com/questions/28194703/recursive-xml-parsing-python-using-elementtree
indent = 0
def printRecur(root):
    global indent
    """Recursively prints the tree."""
    print(' '*indent + '%s: %s' % (root.tag, root.attrib.get('name', root.text)))
    indent += 4
    for elem in root.getchildren():
        printRecur(elem)
    indent -= 4

'''

'\n# check structure of retrived data\n# https://stackoverflow.com/questions/28194703/recursive-xml-parsing-python-using-elementtree\nindent = 0\ndef printRecur(root):\n    global indent\n    """Recursively prints the tree."""\n    print(\' \'*indent + \'%s: %s\' % (root.tag, root.attrib.get(\'name\', root.text)))\n    indent += 4\n    for elem in root.getchildren():\n        printRecur(elem)\n    indent -= 4\n\n'

# Queries

- QUERIES, TARGETS, FILTERS: list of string(s) separated by ' , '; <br>
    search terms are generated by combination of each term from {QUERIES} and {TARGETS} and {FILTERS}<br>
    ex. <br>
    QUERIES = "Ginkgo, CoQ10"<br>
    TARGETS = "hearing loss, tinnitus"<br>
    FILTERS = "clinical trial[pt], animal[Filter]"
- OUT_XLSX_NAME: filename of the output xlsx
    


In [ ]:
SEARCH_URL = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
FETCH_URL = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
SOURCE_DB = 'pubmed'

QUERIES = "Ginkgo, CoQ10, N-acetyl-cysteine, melatonin, glucocorticoid" #@param {type:"string"}
QUERIES = QUERIES.replace(', ', ',').split(',')
TARGETS = "hearing loss" #@param {type:"string"}
TARGETS = TARGETS.replace(', ', ',').split(',')
FILTERS = "clinical trial[pt], animal[Filter]" #@param {type:"string"}
FILTERS = FILTERS.replace(', ', ',').split(',')
OUT_XLSX_NAME = 'Some_PubMed_Summary'#@param {type:"string"}

# Fetch

In [ ]:
os.makedirs('/content/OUTPUT/', exist_ok = True)
with pd.ExcelWriter(f'/content/OUTPUT/{OUT_XLSX_NAME}.xlsx', mode = 'w') as writer:
    for t in TARGETS:
        for q in QUERIES:
            for f in FILTERS:
                res_df = pullData(f'{t} AND {q} AND {f}')
                # remove filter tag
                f = f[:f.find('[')]
                res_df.to_csv(f'/content/OUTPUT/{t}_{q}_{f}.csv', index = False, encoding='utf_8_sig')
                res_df.to_excel(writer, sheet_name = f'{t}_{q}_{f}', index = False, encoding='utf_8_sig')

query translation:
("hearing loss"[MeSH Terms] OR ("hearing"[All Fields] AND "loss"[All Fields]) OR "hearing loss"[All Fields]) AND ("ginkgo biloba"[MeSH Terms] OR ("ginkgo"[All Fields] AND "biloba"[All Fields]) OR "ginkgo biloba"[All Fields] OR "ginkgo"[All Fields]) AND clinical trial[pt]
17 results found


/usr/local/lib/python3.7/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


query translation:
("hearing loss"[MeSH Terms] OR ("hearing"[All Fields] AND "loss"[All Fields]) OR "hearing loss"[All Fields]) AND ("ginkgo biloba"[MeSH Terms] OR ("ginkgo"[All Fields] AND "biloba"[All Fields]) OR "ginkgo biloba"[All Fields] OR "ginkgo"[All Fields]) AND animal[Filter]
22 results found
query translation:
("hearing loss"[MeSH Terms] OR ("hearing"[All Fields] AND "loss"[All Fields]) OR "hearing loss"[All Fields]) AND ("coenzyme Q10"[Supplementary Concept] OR "coenzyme Q10"[All Fields] OR "coq10"[All Fields] OR "ubiquinone"[MeSH Terms] OR "ubiquinone"[All Fields]) AND clinical trial[pt]
4 results found
query translation:
("hearing loss"[MeSH Terms] OR ("hearing"[All Fields] AND "loss"[All Fields]) OR "hearing loss"[All Fields]) AND ("coenzyme Q10"[Supplementary Concept] OR "coenzyme Q10"[All Fields] OR "coq10"[All Fields] OR "ubiquinone"[MeSH Terms] OR "ubiquinone"[All Fields]) AND animal[Filter]
22 results found
query translation:
("hearing loss"[MeSH Terms] OR ("hearing

In [ ]:
!zip -j -r /content/output.zip /content/OUTPUT/

files.download("/content/output.zip")

  adding: hearing loss_CoQ10_animal.csv (deflated 66%)
  adding: hearing loss_melatonin_animal.csv (deflated 65%)
  adding: hearing loss_glucocorticoid_clinical trial.csv (deflated 71%)
  adding: hearing loss_CoQ10_clinical trial.csv (deflated 64%)
  adding: hearing loss_N-acetyl-cysteine_clinical trial.csv (deflated 65%)
  adding: hearing loss_glucocorticoid_animal.csv (deflated 70%)
  adding: hearing loss_N-acetyl-cysteine_animal.csv (deflated 69%)
  adding: PosCont_Supplement_PubMed_Summary.xlsx (deflated 0%)
  adding: hearing loss_Ginkgo_animal.csv (deflated 66%)
  adding: hearing loss_Ginkgo_clinical trial.csv (deflated 67%)
  adding: hearing loss_melatonin_clinical trial.csv (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title
'''
# debugging

search_params = {
"db": SOURCE_DB,
"term": '"hearing loss" AND CoQ10 AND (animal[Filter])',
"usehistory": "y",
"retmode" : "xml",
}
    
root_search = getXMLfromURL(SEARCH_URL, params = search_params)

print(f'query translation:')
print(f'{root_search.find("QueryTranslation").text}')
print(f'{root_search.find("Count").text} results found')

fetch_params = {
    "db" : "pubmed",
    "query_key" : root_search.findtext("QueryKey"),
    "WebEnv" : root_search.findtext("WebEnv"),
    "retmode" : "xml",
}

res_text = requests.get(FETCH_URL, params = fetch_params).text

ignore_tags = ['i', 'b']
for ignore_tag in ignore_tags:
    res_text = \
        res_text.replace('<'+ignore_tag+'>', '').replace('</'+ignore_tag+'>', '')

root = et.fromstring(res_text.encode("utf-8"))
res_dict = parseData(root)
res_df = pd.DataFrame.from_dict(res_dict)
'''

'\n# debugging\n\nsearch_params = {\n"db": SOURCE_DB,\n"term": \'"hearing loss" AND CoQ10 AND (animal[Filter])\',\n"usehistory": "y",\n"retmode" : "xml",\n}\n    \nroot_search = getXMLfromURL(SEARCH_URL, params = search_params)\n\nprint(f\'query translation:\')\nprint(f\'{root_search.find("QueryTranslation").text}\')\nprint(f\'{root_search.find("Count").text} results found\')\n\nfetch_params = {\n    "db" : "pubmed",\n    "query_key" : root_search.findtext("QueryKey"),\n    "WebEnv" : root_search.findtext("WebEnv"),\n    "retmode" : "xml",\n}\n\nres_text = requests.get(FETCH_URL, params = fetch_params).text\n\nignore_tags = [\'i\', \'b\']\nfor ignore_tag in ignore_tags:\n    res_text =         res_text.replace(\'<\'+ignore_tag+\'>\', \'\').replace(\'</\'+ignore_tag+\'>\', \'\')\n\nroot = et.fromstring(res_text.encode("utf-8"))\nres_dict = parseData(root)\nres_df = pd.DataFrame.from_dict(res_dict)\n'

# Reference

- [PubMed APIの概要、データ取得、シンプルなEDA、及びクラスタリング(Python)](https://qiita.com/iwashi-kun/items/bd0d772c6db0c0023e30#:~:text=PubMed%20API%E3%81%A8%E3%81%AF&text=NCBI%E3%81%8C%E9%96%8B%E7%99%BA%E3%83%BB%E9%81%8B%E7%94%A8%E3%81%97,%E3%81%A8%E3%81%97%E3%81%A6%E6%8F%90%E4%BE%9B%E3%81%95%E3%82%8C%E3%81%A6%E3%81%84%E3%81%BE%E3%81%99%E3%80%82)
- [PubMed APIのまとめ](https://yubais.net/doc/pubmed_api/)
- [【PubMed API】Entrez Programming Utilities の使い方](https://www.y-shinno.com/pubmed-api/#toc6)
- [PubMed API Python script](https://nbviewer.org/github/tatsuya-takahashi/PubMed-API-Script/blob/master/PubMed.ipynb)
- [PubMed の使い方](https://www.lib.m.u-tokyo.ac.jp/manual/pubmedmanual.pdf)
- [The E-utilities In-Depth: Parameters, Syntax and More. 2009 May 29 [Updated 2021 Apr 15]. In: Entrez Programming Utilities Help [Internet]. ](https://www.ncbi.nlm.nih.gov/books/NBK25499/)